# Assignment 1: Star Schema/ Snowflake Schema

1)Created required dimension tables(Products, Brand, Categories, ProductVariants, Customers, Addresses, Prices)
2)Created Sales fact table
3)To accommodate the changing nature of customer details, product variants, and prices over time while retaining historical information i adjusted our schema to supprot SCD
and add additional tables for product variants and price history
4)Created below script to create and insert sample data using faker function to generate random fake products for analysis
5)Stored the schema and data to Mysql Database

In [20]:
import mysql.connector
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker to create fake data
fake = Faker()

# Connect to MySQL database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="Nishant@9372",
    database="REUNION2"
)
cursor = conn.cursor()

# Create dimension tables
cursor.execute('''CREATE TABLE IF NOT EXISTS CategoriesDim (
                    CategoryID INT AUTO_INCREMENT PRIMARY KEY,
                    CategoryName VARCHAR(255)
                )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS ProductsDim (
                    ProductID INT AUTO_INCREMENT PRIMARY KEY,
                    ProductName VARCHAR(255),
                    Description TEXT,
                    CategoryID INT,
                    FOREIGN KEY (CategoryID) REFERENCES CategoriesDim(CategoryID)
                )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS ProductVariantsDim (
                    VariantID INT AUTO_INCREMENT PRIMARY KEY,
                    ProductID INT,
                    VariantName VARCHAR(255),
                    Attributes TEXT,
                    CurrentPriceID INT,
                    LaunchDate DATE,
                    DiscontinueDate DATE,
                    CurrentFlag BOOLEAN,
                    FOREIGN KEY (ProductID) REFERENCES ProductsDim(ProductID)
                )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS CustomersDim (
                    CustomerID INT AUTO_INCREMENT PRIMARY KEY,
                    FirstName VARCHAR(255),
                    LastName VARCHAR(255),
                    Gender VARCHAR(50),
                    DateOfBirth DATE,
                    Email VARCHAR(255),
                    Phone VARCHAR(50),
                    CurrentAddressID INT,
                    StartDate DATE,
                    EndDate DATE,
                    CurrentFlag BOOLEAN
                )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS AddressesDim (
                    AddressID INT AUTO_INCREMENT PRIMARY KEY,
                    CustomerID INT,
                    StreetAddress VARCHAR(255),
                    City VARCHAR(255),
                    State VARCHAR(255),
                    Country VARCHAR(255),
                    ZipCode VARCHAR(20),
                    StartDate DATE,
                    EndDate DATE,
                    FOREIGN KEY (CustomerID) REFERENCES CustomersDim(CustomerID)
                )''')



# Create Fact table
cursor.execute('''CREATE TABLE IF NOT EXISTS OrdersFact (
                    OrderID INT AUTO_INCREMENT PRIMARY KEY,
                    CustomerID INT,
                    ProductVariantID INT,
                    OrderDate DATE,
                    OrderQuantity INT,
                    TotalPrice DECIMAL(10, 2),
                    FOREIGN KEY (CustomerID) REFERENCES CustomersDim(CustomerID),
                    FOREIGN KEY (ProductVariantID) REFERENCES ProductVariantsDim(VariantID)
                )''')

# Commit changes and close connection
conn.commit()
conn.close()

print("Tables created successfully.")


Tables created successfully.


In [21]:
cursor = conn.cursor()

# Generate and insert sample data for CategoriesDim
categories = ['Electronics', 'Clothing', 'Home Appliances']
for category in categories:
    cursor.execute('''INSERT INTO CategoriesDim (CategoryName) VALUES (%s)''', (category,))

# Generate and insert sample data for BrandsDim
brands = ['Samsung', 'Apple', 'Nike', 'Adidas', 'Sony']
for brand in brands:
    cursor.execute('''INSERT INTO BrandsDim (BrandName) VALUES (%s)''', (brand,))

# Generate and insert sample data for ProductsDim
products = []
for _ in range(10):
    product_name = fake.word()
    description = fake.text()
    category_id = random.randint(1, len(categories))
    brand_id = random.randint(1, len(brands))
    cursor.execute('''INSERT INTO ProductsDim (ProductName, Description, CategoryID, BrandID) VALUES (%s, %s, %s, %s)''',
                   (product_name, description, category_id, brand_id))
    products.append(cursor.lastrowid)

# Generate and insert sample data for ProductVariantsDim
for product_id in products[:5]:
    for _ in range(2):
        variant_name = fake.word()
        attributes = fake.words(nb=5)
        launch_date = fake.date_between(start_date='-2y', end_date='today')
        discontinue_date = fake.date_between(start_date=launch_date, end_date='today')
        cursor.execute('''INSERT INTO ProductVariantsDim (ProductID, VariantName, Attributes, LaunchDate, DiscontinueDate) VALUES (%s, %s, %s, %s, %s)''',
                       (product_id, variant_name, ' '.join(attributes), launch_date, discontinue_date))

# Generate and insert sample data for CustomersDim
for _ in range(10):
    first_name = fake.first_name()
    last_name = fake.last_name()
    gender = random.choice(['Male', 'Female'])
    dob = fake.date_of_birth(minimum_age=18, maximum_age=90)
    email = fake.email()
    phone = fake.phone_number()
    cursor.execute('''INSERT INTO CustomersDim (FirstName, LastName, Gender, DateOfBirth, Email, Phone) VALUES (%s, %s, %s, %s, %s, %s)''',
                   (first_name, last_name, gender, dob, email, phone))

# Generate and insert sample data for AddressesDim
for customer_id in range(1, 11):
    street_address = fake.street_address()
    city = fake.city()
    state = fake.state()
    country = fake.country()
    zip_code = fake.zipcode()
    start_date = fake.date_between(start_date='-2y', end_date='today')
    end_date = fake.date_between(start_date=start_date, end_date='today')
    cursor.execute('''INSERT INTO AddressesDim (CustomerID, StreetAddress, City, State, Country, ZipCode, StartDate, EndDate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)''',
                   (customer_id, street_address, city, state, country, zip_code, start_date, end_date))

# Generate and insert sample data for PricesDim
for product_id in products[:5]:
    for _ in range(2):
        variant_id = random.randint(1, 10)
        price = round(random.uniform(50, 1000), 2)
        effective_date = fake.date_between(start_date='-2y', end_date='today')
        end_date = fake.date_between(start_date=effective_date, end_date='today')
        cursor.execute('''INSERT INTO PricesDim (VariantID, Price, EffectiveDate, EndDate) VALUES (%s, %s, %s, %s)''',
                       (variant_id, price, effective_date, end_date))

# Generate and insert sample data for OrdersFact
for _ in range(500):  # 500 orders for 2 years of order history
    customer_id = random.randint(1, 10)
    product_variant_id = random.randint(1, 10)
    order_date = fake.date_between(start_date='-2y', end_date='today')
    order_quantity = random.randint(1, 5)
    total_price = round(random.uniform(10, 500), 2)
    cursor.execute('''INSERT INTO OrdersFact (CustomerID, ProductVariantID, OrderDate, OrderQuantity, TotalPrice)
                      VALUES (%s, %s, %s, %s, %s)''',
                   (customer_id, product_variant_id, order_date, order_quantity, total_price))

# Commit changes and close connection
conn.commit()
conn.close()

print("Sample data inserted successfully.")

OperationalError: MySQL Connection not available.

# Assignment 2:SQL

By taking reference of above star schema created above performed all the required query in Mysql console

In [ ]:
#1) Retrieve the top 5 customers who have made the highest average order amounts in the last 6 months. The average order amount should be calculated for each customer, and the result should be sorted in descending order.
SELECT O.CustomerID, C.FirstName, AVG(O.TotalPrice) AS AverageOrderAmount
FROM OrdersFact O
Left Join customersdim C on O.CustomerID=C.CustomerID
WHERE OrderDate >= DATE_SUB(CURRENT_DATE(), INTERVAL 6 MONTH)
GROUP BY CustomerID
ORDER BY AverageOrderAmount DESC
LIMIT 5;

In [ ]:
#2)Retrieve the list of customer whose order value is lower this year as compared to previous year
WITH cte1 AS (
    SELECT CustomerID, SUM(TotalPrice) AS TotalOrderSum2024
    FROM ordersfact
    WHERE YEAR(OrderDate) = 2024
    GROUP BY CustomerID
),
cte2 AS (
    SELECT CustomerID, SUM(TotalPrice) AS TotalOrderSum2023
    FROM ordersfact
    WHERE YEAR(OrderDate) = 2023
    GROUP BY CustomerID
)
SELECT cte1.CustomerID, c.FIRSTNAME
FROM cte1        
LEFT JOIN cte2 ON cte1.CustomerID = cte2.CustomerID
LEFT JOIN CustomersDim c on c.CustomerID = cte1.CustomerID
WHERE cte2.TotalOrderSum2023 > cte1.TotalOrderSum2024;

In [ ]:
#3) Create a table showing cumulative purchase by a particular customer. Show the breakup of cumulative purchases by product category
SELECT 
    c.CustomerID,
    CONCAT(c.FirstName, ' ', c.LastName) AS CustomerName,
    p.CategoryID,
    cd.CategoryName,
    SUM(o.TotalPrice) AS CumulativePurchase
FROM 
    CustomersDim c
JOIN 
    OrdersFact o ON c.CustomerID = o.CustomerID
JOIN 
    ProductVariantsDim pv ON o.ProductVariantID = pv.VariantID
JOIN 
    ProductsDim p ON pv.ProductID = p.ProductID
JOIN 
    CategoriesDim cd ON p.CategoryID = cd.CategoryID
WHERE 
    c.CustomerID = 1
GROUP BY 
    c.CustomerID, p.CategoryID
ORDER BY 
    c.CustomerID, p.CategoryID;

In [ ]:
#4) Retrieve the list of top 5 selling products. Further bifurcate the sales by product variants
SELECT
    p.ProductID,
    p.ProductName,
    pv.VariantID,
    pv.VariantName,
    SUM(o.OrderQuantity) AS TotalQuantity
FROM
    ProductsDim p
JOIN
    ProductVariantsDim pv ON p.ProductID = pv.ProductID
JOIN
    OrdersFact o ON pv.VariantID = o.ProductVariantID
GROUP BY
    p.ProductID, p.ProductName, pv.VariantID, pv.VariantName
ORDER BY
    SUM(o.OrderQuantity) DESC
LIMIT 5;

# Assignment 5: Time Series

Python Solution

In [ ]:
import pandas as pd
from collections import defaultdict

# Load the Excel file
df = pd.read_excel('Time Series.xlsx')

# Convert Start and End columns to datetime
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

# Correct the End dates if the End date is earlier than the Start date
df.loc[df['End'] < df['Start'], 'End'] += pd.DateOffset(years=1)

# Function to merge overlapping periods and aggregate activities for a single bot
def merge_periods_and_aggregate_activities(bot_df):
    bot_df = bot_df.sort_values(by='Start').reset_index(drop=True)
    
    merged_periods = []
    current_start = bot_df.loc[0, 'Start']
    current_end = bot_df.loc[0, 'End']
    current_activities = [bot_df.loc[0, 'Activity']]
    
    for i in range(1, len(bot_df)):
        row = bot_df.loc[i]
        if row['Start'] <= current_end:
            current_end = max(current_end, row['End'])
            current_activities.append(row['Activity'])
        else:
            merged_periods.append((current_start, current_end, current_activities))
            current_start = row['Start']
            current_end = row['End']
            current_activities = [row['Activity']]
    
    merged_periods.append((current_start, current_end, current_activities))
    
    return merged_periods

# Group by bot name and process each group
result = defaultdict(list)
for name, group in df.groupby('Name'):
    result[name] = merge_periods_and_aggregate_activities(group)

# Convert the result to a structured format
output = []
for name, periods in result.items():
    for start, end, activities in periods:
        output.append({'Name': name, 'Start': start, 'End': end, 'Activities': activities})

# Convert the output to a DataFrame
output_df = pd.DataFrame(output)

# Save to a new Excel file
output_df.to_excel('output.xlsx', index=False)


SQL Solution

In [ ]:
WITH RECURSIVE WorkPeriods AS (
    SELECT 
        Name,
        Start,
        End,
        Activity,
        ROW_NUMBER() OVER (PARTITION BY Name ORDER BY Start) AS rn
    FROM agi_tasks
),
MergedPeriods AS (
    SELECT
        wp1.Name,
        wp1.Start,
        wp1.End,
        wp1.Activity
    FROM WorkPeriods wp1
    WHERE wp1.rn = 1
    UNION ALL
    SELECT
        wp2.Name,
        LEAST(mp.Start, wp2.Start),
        GREATEST(mp.End, wp2.End),
        mp.Activity || ', ' || wp2.Activity
    FROM MergedPeriods mp
    JOIN WorkPeriods wp2 ON mp.Name = wp2.Name AND wp2.rn = (SELECT rn + 1 FROM WorkPeriods WHERE Name = wp2.Name AND Start = mp.Start)
    WHERE wp2.Start <= mp.End
),
FinalPeriods AS (
    SELECT
        Name,
        Start,
        End,
        Activity,
        ROW_NUMBER() OVER (PARTITION BY Name, Start ORDER BY End DESC) AS rn
    FROM MergedPeriods
)
SELECT 
    Name,
    MIN(Start) AS Start,
    MAX(End) AS End,
    Activity
FROM FinalPeriods
WHERE rn = 1
GROUP BY Name, Activity
ORDER BY Name, Start;


# Assignment 4: Streaming

1)Firstly Created Azure storage Account
2)Navigated to storage account and created new container
3)Created a Databricks workspace and written the below code at notebook
4)The below code writes the ingested data directly to a Delta table in the specified Azure Blob Storage path without performing any transformations.
5)The cloudFiles.inferColumnTypes option in Autoloader automatically infers the schema of the ingested data.
6)Databricks Autoloader automatically detects new files added to the specified Azure Blob Storage path. 

In [18]:
pip install pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession


# Set up the configurations
storage_account_name = "newnishantstorageaccount"
container_name = "reunionassignment"
sas_token = "sp=r&st=2024-06-13T07:08:21Z&se=2024-06-13T15:08:21Z&spr=https&sv=2022-11-02&sr=c&sig=BIXwpx9R0JOdeA2P%2FI4hS1s16KLC%2FPXDnCayKkQZNKY%3D"
# Define the input and schema paths
input_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/?{sas_token}"
schema_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/schema"

# Configure Autoloader
df = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "json") \
    .option("cloudFiles.inferColumnTypes", "true") \
    .option("cloudFiles.schemaLocation", schema_path) \
    .load(input_path)

# Write data to a Delta table
output_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/output_table"

query = df.writeStream \
    .format("delta") \
    .option("checkpointLocation", f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/checkpoints") \
    .start(output_path)

query.awaitTermination()


SyntaxError: invalid syntax (3341294003.py, line 1)

# Assignment 3: ETL

In [ ]:
import json
import pandas as pd
from sqlalchemy import create_engine, text
import urllib.parse

# Load JSON data
with open('Reunion_nested_data.json') as f:
    data = json.load(f)

# Initialize lists for each table
programs_list = []
concerts_list = []
works_list = []
soloists_list = []
composers_set = set()

# Parse the JSON data and populate the lists
for program in data['programs']:
    program_id = program['programID']
    season = program['season']
    orchestra = program['orchestra']
    
    programs_list.append({'ProgramID': program_id, 'Season': season, 'Orchestra': orchestra})
    
    for concert in program['concerts']:
        concerts_list.append({'ProgramID': program_id, 'Date': concert['Date'], 'EventType': concert['eventType'], 'Venue': concert['Venue'], 'Location': concert['Location'], 'Time': concert['Time']})
    
    for work in program['works']:
        work_id = work['ID']
        title = work.get('workTitle', None)
        conductor = work.get('conductorName', None)
        composer = work.get('composerName', None)  # Use .get() to handle missing 'composerName'
        movement = work.get('movement', None)
        interval = work.get('interval', None)
        
        works_list.append({'WorkID': work_id, 'ProgramID': program_id, 'Title': title, 'ConductorName': conductor, 'Composer': composer, 'Movement': movement, 'Interval': interval})
        
        # Add composer to the set
        if composer:
            composers_set.add(composer)
        
        for soloist in work.get('soloists', []):  # Use .get() to handle missing 'soloists'
            soloists_list.append({'SoloistName': soloist['soloistName'], 'Roles': soloist['soloistRoles'], 'Instrument': soloist['soloistInstrument'], 'WorkID': work_id})

# Convert the sets to lists
composers_list = [{'Name': composer} for composer in composers_set]

# Create dataframes
programs_df = pd.DataFrame(programs_list)
concerts_df = pd.DataFrame(concerts_list)
works_df = pd.DataFrame(works_list)
soloists_df = pd.DataFrame(soloists_list)
composers_df = pd.DataFrame(composers_list)

# MySQL connection details
mysql_user = 'root'
mysql_password = urllib.parse.quote_plus('Nishant@9372')  # Encode the password
mysql_host = '127.0.0.1'
mysql_db = 'orchestra'

# Create SQLAlchemy engine
engine = create_engine(f'mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_db}')

# Insert data into works table row by row
for index, row in works_df.iterrows():
    insert_query = "INSERT INTO works (`WorkID`, `ProgramID`, `Title`, `ConductorName`, `Composer`, `Movement`, `Interval`) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    engine.execute(insert_query, tuple(row))

# Save other dataframes to the MySQL database
programs_df.to_sql('programs', engine, if_exists='replace', index=False)
concerts_df.to_sql('concerts', engine, if_exists='replace', index=False)
soloists_df.to_sql('soloists', engine, if_exists='replace', index=False)
composers_df.to_sql('composers', engine, if_exists='replace', index=False)

# Print DataFrames for verification
print(programs_df)
print(concerts_df)
print(soloists_df)
print(composers_df)
